<img align="center" src="data/handout/course.png" width="800">

## Hough Transform - Assignment 1
    Instructor: Kris                          TAs: Arka, Jinkun, Rawal, Rohan, Sheng-Yu

In this assignment you will be implementing a Hough Transform based line detector. 

<img align="center" src="data/handout/ht.gif" width="500">

---
## Theory Questions (25 points)
**Grading**: 
- Each question is 5 points. 
- Please add your answers to the writeup. Insert images whenever necessary.
- Show all your work to obtain full credit.

**Q1**: Show that using $ x\cos \theta + y \sin \theta - \rho = 0 $, each image point $(x, y)$ results in a sinsuoid in $(\rho, \theta)$ hough space.  
Use the formulation, $a \sin \theta + b \cos \theta = A \sin(\theta + \phi)$ for the sinsuoid.  
Write the amplitude \\(A \\) and phase \\( \phi \\) of the sinusoid as a function of \\( (x, y) \\). 

YOUR ANSWER HERE

**Q2**: Why do we parameterize the line in terms of \\(\rho, \theta\\) instead of slope and intercept \\((m, c)\\)?  
Also write the slope \\(m\\) and intercept \\(c\\) as a function of \\(\rho\\) and \\(\theta\\). 

YOUR ANSWER HERE

**Q3**: Assume the image points \\((x, y)\\) are in an image of width \\(W\\) and height \\(H\\), \\(x \in [1, W], y \in [1, H]\\).  
What is the maximum absolute value of \\(\rho\\) and what is the range of \\(\theta\\)?

YOUR ANSWER HERE

**Q4**: For points $(10, 10)$, $(15, 15)$ and $(30, 30)$ in the image, plot the corresponding sinusoid waves in Hough space $(\rho, \theta)$.  
Also visualize how their intersection point defines the line (what is ($m, c$) for this line?).  
Please add the plot as image.

YOUR ANSWER HERE

**Q5**: How does the dimension of parameter space affects Hough Transform method? What would you do when the parameter space is high, i.e., 3D or 4D instead of 2D? Briefly explain your method.

YOUR ANSWER HERE